# Module 3: Batch Scoring using a pre-trained XGBoost model
**This notebook uses the feature set extracted by `module-2` to create a XGBoost based machine learning model for binary classification**

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.t3.medium`

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Prepare test data](#Prepare-test-data)
1. [Batch Transform](#Batch-Transform)

## Background

After the model is trained, if the goal is to generate predictions on a large dataset where minimizing latency isn't a concern, then SageMaker batch transform is the solution. Functionally, batch transform uses the same mechanics as real-time hosting to generate predictions. It requires a web server that takes in HTTP POST requests a single observation, or mini-batch, at a time. However, unlike real-time hosted endpoints which have persistent hardware (instances stay running until you shut them down), batch transform clusters are torn down when the job completes.

In this example, we will walk through the steps to prepare the batch test dataset from feature store using processing job and perform batch transform with the test data available on Amazon S3. 

## Setup


In [ ]:
import logging
import sys
from urllib.parse import urlparse

import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.dataset_definition.inputs import (
    AthenaDatasetDefinition,
    DatasetDefinition,
)
from sagemaker.model import Model
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.utils import name_from_base

sys.path.append("..")
from utilities import Utils
import uuid
import time
from time import sleep

In [ ]:
logger = logging.getLogger("__name__")
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Essentials

In [ ]:
sagemaker_execution_role = get_execution_role()
logger.info(f"Role = {sagemaker_execution_role}")
session = boto3.Session()
sagemaker_session = sagemaker.Session()
sagemaker_client = session.client(service_name="sagemaker")


default_bucket = sagemaker_session.default_bucket()
prefix = "sagemaker-featurestore-workshop"
s3 = session.resource("s3")

### Prepare test data for batch transform 
<!-- job using processing job with *AthenaDatasetDefinition* -->
We create the test dataset that we use in our batch transform job using [*AthenaDatasetDefinition*](https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html#sagemaker.dataset_definition.inputs.AthenaDatasetDefinition) API.
We follow the steps below to prepare the test dataset for batch transform job:
1. firstly generates the list of feature names that we would like to read from the offline feature store by providing the feature group names as a list and an exclude feature list to the *generate_fsets* function. 
2. Construct an Athena query to read the data from offline feature store and run a SageMaker processing job to transform the data type to 'text/csv'.

#### Generate the list of features needed from feature store.

We use boto3 sagemaker_client to perform `DescribeFeatureGroup` action to describe a FeatureGroup. The response includes information on the creation time, FeatureGroup name, the unique identifier for each FeatureGroup, and more, for more details of the response syntax, please refer to [document here](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeFeatureGroup.html#API_DescribeFeatureGroup_ResponseSyntax).

In [ ]:
# Retreive FG names
%store -r customers_feature_group_name
%store -r products_feature_group_name
%store -r orders_feature_group_name

customers_fg = sagemaker_client.describe_feature_group(
    FeatureGroupName=customers_feature_group_name
)
products_fg = sagemaker_client.describe_feature_group(
    FeatureGroupName=products_feature_group_name
)
orders_fg = sagemaker_client.describe_feature_group(
    FeatureGroupName=orders_feature_group_name
)

database_name = customers_fg["OfflineStoreConfig"]["DataCatalogConfig"]["Database"]
catalog = customers_fg["OfflineStoreConfig"]["DataCatalogConfig"]["Catalog"]

customers_table = customers_fg["OfflineStoreConfig"]["DataCatalogConfig"]["TableName"]
products_table = products_fg["OfflineStoreConfig"]["DataCatalogConfig"]["TableName"]
orders_table = orders_fg["OfflineStoreConfig"]["DataCatalogConfig"]["TableName"]

In [ ]:
exclude_fsets = [
    "customer_id",
    "product_id",
    "order_id",
    "event_time",
    "purchase_amount",
    "n_days_since_last_purchase",
]

In [ ]:
def generate_fsets(fg_list, exclude_fsets=None):
    _fg_lst = []
    for _fg in fg_list:
        _fg_tmp = pd.DataFrame(
            Utils.describe_feature_group(_fg["FeatureGroupName"])["FeatureDefinitions"]
        )
        if exclude_fsets:
            _fg_tmp = _fg_tmp[~_fg_tmp.FeatureName.isin(exclude_fsets)]

        _fg_lst.append(_fg_tmp)
    return pd.concat(_fg_lst, ignore_index=True)

In [ ]:
fsets_df = generate_fsets([orders_fg, customers_fg, products_fg], exclude_fsets)
features_names = fsets_df.FeatureName.tolist()

#### Use Athena CTAS table query to generate test set for batch job



We start by create an Athena query to get the test data from feature store. Note that the first column will be the unique identifier of customer id and the second column is the target value. Note that the query should only take the latest version of any given record that has multiple write times for the same event_time.

In [ ]:
batch_transform_columns_string = ",\n    ".join(f'"{c}"' for c in features_names)

customer_uid = customers_fg["RecordIdentifierFeatureName"]
product_uid = products_fg["RecordIdentifierFeatureName"]
order_uid = orders_fg["RecordIdentifierFeatureName"]

customer_et = customers_fg["EventTimeFeatureName"]
product_et = products_fg["EventTimeFeatureName"]
order_et = orders_fg["EventTimeFeatureName"]


query_string = f"""WITH customer_table AS (
    SELECT *,
        dense_rank() OVER (
            PARTITION BY "{customer_uid}"
            ORDER BY "{customer_et}" DESC,
                "api_invocation_time" DESC,
                "write_time" DESC
        ) AS "rank"
    FROM "{customers_table}"
    WHERE NOT "is_deleted"
),
product_table AS (
    SELECT *,
        dense_rank() OVER (
            PARTITION BY "{product_uid}"
            ORDER BY "{product_et}" DESC,
                "api_invocation_time" DESC,
                "write_time" DESC
        ) AS "rank"
    FROM "{products_table}"
    WHERE NOT "is_deleted"
),
order_table AS (
    SELECT *,
        dense_rank() OVER (
            PARTITION BY "{order_uid}"
            ORDER BY "{order_et}" DESC,
                "api_invocation_time" DESC,
                "write_time" DESC
        ) AS "rank"
    FROM "{orders_table}"
    WHERE NOT "is_deleted"
)
SELECT DISTINCT
    "{order_uid}",
    {batch_transform_columns_string}
FROM customer_table,
    product_table,
    order_table
WHERE order_table."customer_id" = customer_table."customer_id"
    AND order_table."product_id" = product_table."product_id"
    AND customer_table."rank" = 1
    AND product_table."rank" = 1
    AND order_table."rank" = 1
"""
# print(query_string)

In [ ]:
destination_s3_path = f's3://{default_bucket}/{prefix}/athena/data/sagemaker-batch-{time.strftime("%Y-%m-%d-%H%M%S")}/data'
table_name = f'{catalog}.{database_name}.sagemaker_tmp_{uuid.uuid4().hex[:8]}'

CTAS_query = f"""CREATE TABLE {table_name} WITH 
(external_location='{destination_s3_path}', format='PARQUET') 
AS {query_string}
"""
print(CTAS_query)

In [ ]:
athena = boto3.client('athena')

tmp_uri = f's3://{default_bucket}/{prefix}/offline-store/query_results/'
print('Running query on  database: ' + database_name)
query_execution = athena.start_query_execution(
    QueryString=CTAS_query,
    QueryExecutionContext={'Database': database_name},
    ResultConfiguration={'OutputLocation': tmp_uri}
)
# wait for the Athena query to complete
query_execution_id = query_execution['QueryExecutionId']
query_state = athena.get_query_execution(QueryExecutionId=query_execution_id)['QueryExecution']['Status']['State']
while (query_state != 'SUCCEEDED' and query_state != 'FAILED'):
    sleep(2)
    query_state = athena.get_query_execution(QueryExecutionId=query_execution_id)['QueryExecution']['Status']['State']

if query_state == 'FAILED':
    print(athena.get_query_execution(QueryExecutionId=query_execution_id))
    failure_reason = athena.get_query_execution(QueryExecutionId=query_execution_id)['QueryExecution']['Status']['StateChangeReason']
    print(failure_reason)
else:
    !aws s3 ls $destination_s3_path/

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.

#### 1. Create a model based on the pre-trained model artifacts on S3
Let's first create a model based on the training job from the previous notebook. We can use `describe_training_job` boto3 api call to get the model data uri and the container used for the training job. Please note that the pre-built [XGBoost container](https://github.com/aws/sagemaker-xgboost-container) uses the same container image for training and hosting. However, if you are using other framework containers, such as TensorFlow, PyTorch and etc., the training and inference containers are different. For more details about the the available deep learning containers, please refer to the [github page](https://github.com/aws/deep-learning-containers/blob/master/available_images.md). 

In [ ]:
%store -r training_jobName

from sagemaker.xgboost.model import XGBoostModel

training_job_info = sagemaker_client.describe_training_job(
    TrainingJobName=training_jobName
)
xgb_model_data = training_job_info["ModelArtifacts"]["S3ModelArtifacts"]

xgb_model = XGBoostModel(
    source_dir='./code',
    entry_point='inference.py',
    framework_version='1.0-1',
    model_data=xgb_model_data,
    role=sagemaker_execution_role,
    name=name_from_base("fs-workshop-xgboost-model"),
    sagemaker_session=sagemaker_session,
)

In [ ]:
output_path = f's3://{default_bucket}/{prefix}/batch_output/{xgb_model.name}'

xgb_transformer = xgb_model.transformer(
    strategy='SingleRecord',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    max_concurrent_transforms=4,
    accept="text/csv",
    output_path=output_path
)
xgb_transformer.transform(
    destination_s3_path, content_type="application/x-parquet", #wait=False,
)

Let's inspect the output of the Batch Transform job in S3. 

In [ ]:
s3 = boto3.resource("s3")


def list_s3_files(s3uri):
    parsed_url = urlparse(s3uri)
    bucket = s3.Bucket(parsed_url.netloc)
    prefix = parsed_url.path[1:]
    return [
        dict(bucket_name=k.bucket_name, key=k.key)
        for k in bucket.objects.filter(Prefix=prefix)
    ]

In [ ]:
output_file_list = list_s3_files(xgb_transformer.output_path)
output_file_list

In [ ]:
pd.read_csv(
    "s3://{bucket_name}/{key}".format(**output_file_list[0]),
    header=None,
)

#### Clean up


In [ ]:
## TODO
# !aws glue delete-table --database-name $database_name --name $table_name